# Prepare

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime
from scipy.stats import entropy
import scipy as sp

/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import sparse_rail_generator
from flatland.envs.line_generators import sparse_line_generator
from flatland.envs.observations import GlobalObsForRailEnv

In [3]:
rail_generator = sparse_rail_generator(max_num_cities=10)

env = RailEnv(
    width=30, 
    height=30, 
    number_of_agents=1,
    rail_generator=rail_generator,
    line_generator=sparse_line_generator(),
    obs_builder_object=GlobalObsForRailEnv(),
    )
obs = env.reset()
while True:
    obs, rew, done, info = env.step({
            0: np.random.randint(0, 5),
            1: np.random.randint(0, 5)
        })
    if done:
        break

import PIL
from flatland.utils.rendertools import RenderTool
from IPython.display import clear_output


# Render the environment
def render_env(env,wait=True):
    
    env_renderer = RenderTool(env, gl="PILSVG")
    env_renderer.render_env()

    image = env_renderer.get_image()
    pil_image = PIL.Image.fromarray(image)
    clear_output(wait=True)
    display(pil_image)

In [4]:
# Hyperparameters
gamma = 0.9  # Discount factor for past rewards
nb_steps = 100
nb_episodes = 1000
learning_rate = 7e-4
bootstrap_n = 10

beta_v = 0.05
beta_e = 0.05

In [5]:
TRANSITION_MAPS = 0
AGENT_STATES = 1
AGENT_TARGETS = 2

MY_POS_DIR = 0
OTHER_POS_DIR = 1
MALFUNCTIONS = 2
FRAC_SPEEDS = 3
NUM_AGENTS_WITH_ME = 4

NOTHING = 0
LEFT = 1
FOWARD = 2
RIGHT = 3
STOP = 4

In [6]:
# Save Paths
path = "train/" + datetime.now().strftime("%m%d-%H:%M:%S")
log_dir = path+'/logs/'
ckpt_dir = path+'/ckpt/'
train_summary_writer = tf.summary.create_file_writer(log_dir)

2023-01-03 11:05:28.028832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# LOSS

In [7]:
def compute_loss(
        action_probs: tf.Tensor,
        values: tf.Tensor,
        rewards: tf.Tensor,
        entropy : tf.Tensor) -> tf.Tensor:
    """Computes the combined actor-critic loss."""
    
    #bootstrap_n = tf.shape(rewards)[0]
    
    # R_t = []
    # rewards = rewards[::-1]
    
    # discounted_sum = values[-1]
    # for i in tf.range(bootstrap_n):
    #     discounted_sum = rewards[i] + gamma * discounted_sum
    #     R_t.append(discounted_sum)
    
    # #R_t = tf.convert_to_tensor(R_t)
    # R_t = tf.convert_to_tensor(R_t[::-1])
    # delta = R_t - values

    critic_loss = beta_v * 0.5 * tf.reduce_sum(tf.square(values))
    #critic_loss = beta_v * tf.reduce_sum(delta * values)
    
    #log_actor = tf.math.log(tf.reduce_sum(action_probs) + 1e-7)
    #actor_loss = tf.reduce_sum(log_actor * delta)
    actor_loss = -tf.reduce_sum(tf.math.log(action_probs + 1e-7))
    
    entropy = beta_e * entropy

    total_loss = actor_loss + critic_loss + entropy

    return total_loss, actor_loss, critic_loss, entropy

# LSTM

In [8]:
num_inputs = 16200
num_actions = 5
num_hidden = 256

inputs = layers.Input(shape=(num_inputs))
state_h = layers.Input(shape=(num_hidden))
state_c = layers.Input(shape=(num_hidden))

common, states = layers.LSTMCell(num_hidden)(inputs, states=[state_h, state_c], training=True)
action = layers.Dense(num_actions, activation="softmax")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs=[inputs,state_h,state_c], outputs=[action, critic, states])

#model.save('init.h5')

In [9]:
def pepare_obs(next_obs, agent_id):
    my_obs = next_obs.get(agent_id)
    map = my_obs[TRANSITION_MAPS].flatten()
    pos_dir = my_obs[AGENT_STATES].T[MY_POS_DIR].T.flatten()
    target = my_obs[AGENT_TARGETS].T[0].T.flatten()
    
    return np.concatenate((map, pos_dir, target)).reshape(1,16200)

# Train

In [10]:
agent_id = 0
optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
running_reward = 0
episode_count = 0
#model = keras.models.load_model('init.h5')
  

for episode in range(nb_episodes):  # Run until solved
    env.reset()
    action_probs_history = []
    critic_value_history = []
    rewards_history = []
    actions = []
    episode_reward = 0
    reward = 0.0
    action_onehot = np.zeros((num_actions))
    cell_state = [tf.zeros((1,num_hidden)),tf.zeros((1,num_hidden))]
    
    episode_entropy = tf.zeros(())
    
    next_obs, all_rewards, dones, _ = env.step({agent_id: NOTHING})
    
    with tf.GradientTape() as tape:
        for timestep in range(nb_steps):
            
            input = pepare_obs(next_obs, agent_id)

            # Predict action probabilities and estimated future rewards from environment state
            action_probs, critic_value, cell_state = model([input,cell_state[0],cell_state[1]])
            
            critic_value_history.append(tf.squeeze(critic_value))

            # Sample action from action probability distribution
            action_probs = tf.squeeze(action_probs)
            action = np.random.choice(num_actions, p=action_probs.numpy())
            action_probs_history.append(action_probs[action])
            action_onehot[action] = 1
            actions.append(action)

            # Apply the sampled action in our environment
            #state, reward, done, _ = env.trial(action)
            next_obs, all_rewards, dones, _ = env.step({agent_id: action})
            reward = all_rewards[agent_id]
            done = dones[0]
            
            rewards_history.append(reward)
            
            #entropy
            #entropy = -tf.math.reduce_sum(tf.math.multiply(tmp,tf.math.log(tmp + 1e-7)))
            entropy = sp.stats.entropy(action_probs)
            episode_entropy += entropy
            
            if done: break


        # Calculating loss values to update our network
        total_loss, actor_loss, critic_loss, entropy = compute_loss(
            tf.convert_to_tensor(action_probs_history,dtype=tf.float32), 
            tf.convert_to_tensor(critic_value_history, dtype=tf.float32), 
            tf.convert_to_tensor(rewards_history, dtype=tf.float32), 
            episode_entropy)
        
        # Backpropagation
        grads = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        with train_summary_writer.as_default():
            tf.summary.scalar('loss/total_loss', total_loss, step=episode)
            tf.summary.scalar('loss/actor_loss', actor_loss, step=episode)
            tf.summary.scalar('loss/critic_loss', critic_loss, step=episode)
            tf.summary.scalar('loss/entropy', episode_entropy, step=episode)
            tf.summary.scalar('game/reward', reward, step=episode)
            tf.summary.histogram('game/actions', actions, step=episode)
            tf.summary.scalar('game/done', done, step=episode)
            
        print(rewards_history)
            
    # Log details
    if episode % 100 == 0:
        checkpoint = tf.train.Checkpoint(model)
        save_path = checkpoint.save(ckpt_dir+'checkpoints_'+str(episode_count)+'/two_steps.ckpt')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -36]


/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 3/4
  warnings.warn(city_warning)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -79]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -59]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -63]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 2/4
  warnings.warn(city_warning)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -46]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -70]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -73]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

KeyboardInterrupt: 

In [ ]:
model.save(path+'/model.h5')

In [ ]:
%load_ext tensorboard